In [1]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import great_circle
import os

In [2]:
official_filename = "SP_Freshfood_merged.shp"
OSM_filename = "sao_paulo_br_2019_1600m_buffer.gpkg"
POIs_name = "fresh_food_market"

gdf = gpd.read_file(OSM_filename, layer='destinations')
gdf_osm = gdf[gdf['dest_name']==POIs_name]
gdf_official = gpd.GeoDataFrame.from_file(official_filename)
print("number of OSM points: " ,len(gdf_osm))
print("number of Official points: ", len(gdf_official))


number of OSM points:  2132
number of Official points:  939


In [3]:
#Project gdf_osm to the crs of the official layer
gdf_osm = gdf_osm.to_crs(gdf_official.crs)
print(gdf_osm.crs)

epsg:31983


In [4]:
#Write data to shapefile
from os.path import join as pjoin
filename = "sao_paulo_poi_official.shp"
path_to_file = pjoin("output", filename)

if os.path.isfile(path_to_file):
    print ("File exist")
else:
    gdf_official.to_file(path_to_file)
    
filename = "sao_paulo_poi_osm.shp"
path_to_file = pjoin("output", filename)
#gdf_osm = pd.merge(gdf_osm, gdf_osm.bounds, left_index=True, right_index=True)


if os.path.isfile(path_to_file):
    print ("File exist")
else:
    gdf_osm.to_file(path_to_file)


In [5]:
#Calculate number of gdf_osm intersect with gdf_official
mask1 = gdf_osm['geometry'].intersects(gdf_official['geometry'].unary_union)

print("number of intersection items: ",len(gdf_osm[mask1]))
print("Percentage: ",len(gdf_osm[mask1])*100/len(gdf_osm) )

number of intersection items:  0
Percentage:  0.0


In [9]:
#Calculate intersection with various buffers
buff = 5
percent_list = []
percent_dict = {}

while buff<100:
    OSM_buffer = gdf_osm
    OSM_buffer['geometry'] = OSM_buffer.geometry.buffer(buff)
    OP_buffer =gdf_official
    OP_buffer['geometry'] = OP_buffer.geometry.buffer(buff)
    
    print("Buffer: ", str(buff))
    
    #Calculate percent of OSM points within Official points 100m buffer
    mask = gdf_osm['geometry'].intersects(OP_buffer['geometry'].unary_union)
    print("intersection: ",len(gdf_osm[mask]))
    print("percent of OSM points", str(len(gdf_osm[mask])*100/len(gdf_osm)))
    
    #Calculate percent of Official Point within OSM points 100m buffer
    mask = gdf_official['geometry'].intersects(OSM_buffer['geometry'].unary_union)
    print("intersection: ",len(gdf_official[mask]))
    print("percent of Official Point: ", str(len(gdf_official[mask])*100/len(gdf_official)))
    print("/n")
    
    buff+=5
    

Buffer:  5
intersection:  511
percent of OSM points 23.96810506566604
intersection:  219
percent of Official Point:  23.322683706070286
/n
Buffer:  10
intersection:  554
percent of OSM points 25.984990619136962
intersection:  238
percent of Official Point:  25.34611288604899
/n
Buffer:  15
intersection:  638
percent of OSM points 29.924953095684803
intersection:  290
percent of Official Point:  30.8839190628328
/n
Buffer:  20
intersection:  737
percent of OSM points 34.568480300187616
intersection:  332
percent of Official Point:  35.356762513312034
/n
Buffer:  25
intersection:  868
percent of OSM points 40.71294559099437
intersection:  387
percent of Official Point:  41.21405750798722
/n
Buffer:  30
intersection:  999
percent of OSM points 46.85741088180112
intersection:  450
percent of Official Point:  47.92332268370607
/n
Buffer:  35
intersection:  1138
percent of OSM points 53.37711069418386
intersection:  511
percent of Official Point:  54.419595314164006
/n
Buffer:  40
intersecti

In [6]:
#Draw 100m buffer around OSM points; draw 100m buffer around official points
OSM_buffer = gdf_osm
OSM_buffer['geometry'] = OSM_buffer.geometry.buffer(100)
OP_buffer =gdf_official
OP_buffer['geometry'] = OP_buffer.geometry.buffer(100)


In [7]:
#Calculate percent of OSM points within Official points 100m buffer
mask = gdf_osm['geometry'].intersects(OP_buffer['geometry'].unary_union)
print("intersection: ",len(gdf_osm[mask]))
print("percent of OSM points", str(len(gdf_osm[mask])*100/len(gdf_osm)))

intersection:  487
percent of OSM points 22.842401500938088


In [8]:
#Calculate percent of Official Point within OSM points 100m buffer
mask = gdf_official['geometry'].intersects(OSM_buffer['geometry'].unary_union)
print("intersection: ",len(gdf_official[mask]))
print("percent of Official Point: ", str(len(gdf_official[mask])*100/len(gdf_official)))

intersection:  205
percent of Official Point:  21.831735889243877


In [ ]:
#Calculate distance to all Official polygons and pick the min
def min_distance(gdf_osm, gdf_official):
    nearest_dists = []
    for item in gdf_osm.geometry:
        nearest_dist = gdf_official.distance(item).min()
        nearest_dists.append(nearest_dist)
    gdf_osm['nearest_distance'] = nearest_dists
    print(gdf_osm['nearest_distance'])

In [ ]:
min_distance(gdf_osm,gdf_official)
#Graphing the histogram of distance from gdf_osm to nearest Official points
ax = gdf_osm['nearest_distance'].hist()
plt.show()

mean_dist = gdf_osm['nearest_distance'].mean()
print("mean nearest distance: ",mean_dist)

median_dist = gdf_osm['nearest_distance'].median()
print("median nearest distance: ", median_dist)